# ساخت با مدل‌های Mistral

## مقدمه

در این درس به موارد زیر پرداخته می‌شود:
- بررسی مدل‌های مختلف Mistral
- آشنایی با کاربردها و سناریوهای هر مدل
- نمونه کدهایی که ویژگی‌های منحصربه‌فرد هر مدل را نشان می‌دهند.


## مدل‌های میسترال

در این درس، با سه مدل مختلف میسترال آشنا می‌شویم: **Mistral Large**، **Mistral Small** و **Mistral Nemo**.

هر یک از این مدل‌ها به صورت رایگان در بازار مدل‌های گیت‌هاب در دسترس هستند. کد این دفترچه از این مدل‌ها برای اجرای کد استفاده می‌کند. برای اطلاعات بیشتر درباره استفاده از مدل‌های گیت‌هاب جهت [نمونه‌سازی با مدل‌های هوش مصنوعی](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) اینجا را ببینید.


## میسترال لارج ۲ (۲۴۰۷)
میسترال لارج ۲ در حال حاضر مدل پرچم‌دار شرکت میسترال است و برای استفاده سازمانی طراحی شده است.

این مدل نسبت به نسخه اصلی میسترال لارج ارتقا یافته و ویژگی‌های زیر را ارائه می‌دهد:
- پنجره متنی بزرگ‌تر - ۱۲۸ هزار در مقابل ۳۲ هزار
- عملکرد بهتر در وظایف ریاضی و برنامه‌نویسی - دقت متوسط ۷۶.۹٪ در مقابل ۶۰.۴٪
- بهبود عملکرد چندزبانه - زبان‌ها شامل: انگلیسی، فرانسوی، آلمانی، اسپانیایی، ایتالیایی، پرتغالی، هلندی، روسی، چینی، ژاپنی، کره‌ای، عربی و هندی.

با این قابلیت‌ها، میسترال لارج در موارد زیر بسیار موفق عمل می‌کند:
- *تولید تقویت‌شده با بازیابی (RAG)* - به دلیل پنجره متنی بزرگ‌تر
- *فراخوانی تابع* - این مدل به طور بومی از فراخوانی تابع پشتیبانی می‌کند و امکان ادغام با ابزارها و APIهای خارجی را فراهم می‌سازد. این فراخوانی‌ها می‌توانند به صورت موازی یا پشت سر هم انجام شوند.
- *تولید کد* - این مدل در تولید کد برای زبان‌های پایتون، جاوا، تایپ‌اسکریپت و ++C بسیار قدرتمند است.


### نمونه RAG با استفاده از Mistral Large 2


در این مثال، ما از مدل Mistral Large 2 برای اجرای الگوی RAG روی یک سند متنی استفاده می‌کنیم. سؤال به زبان کره‌ای نوشته شده و درباره فعالیت‌های نویسنده قبل از ورود به دانشگاه پرسیده است.

در اینجا از مدل Cohere Embeddings برای ساختن embeddingهای سند متنی و همچنین سؤال استفاده شده است. برای این نمونه، بسته پایتون faiss به عنوان فروشگاه برداری به کار رفته است.

در پرامپتی که به مدل Mistral ارسال می‌شود، هم سؤال و هم بخش‌هایی از متن که بیشترین شباهت را به سؤال دارند، قرار داده می‌شوند. سپس مدل یک پاسخ طبیعی به زبان ارائه می‌دهد.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## میسترال اسمال
میسترال اسمال یکی دیگر از مدل‌های خانواده میسترال است که در دسته مدل‌های پریمیر/سازمانی قرار می‌گیرد. همانطور که از نامش پیداست، این مدل یک مدل زبانی کوچک (SLM) است. مزایای استفاده از میسترال اسمال عبارتند از:
- صرفه‌جویی در هزینه نسبت به مدل‌های زبانی بزرگ میسترال مثل Mistral Large و NeMo - کاهش قیمت تا ۸۰٪
- تأخیر پایین - پاسخ‌دهی سریع‌تر نسبت به مدل‌های زبانی بزرگ میسترال
- انعطاف‌پذیر - قابلیت اجرا در محیط‌های مختلف با محدودیت کمتر در منابع مورد نیاز

میسترال اسمال برای این موارد عالی است:
- کارهای متنی مثل خلاصه‌سازی، تحلیل احساسات و ترجمه
- اپلیکیشن‌هایی که به دلیل مقرون‌به‌صرفه بودن، درخواست‌های مکرر دارند
- کارهای کدنویسی با تأخیر پایین مثل بازبینی و پیشنهاد کد


## مقایسه Mistral Small و Mistral Large

برای مشاهده تفاوت‌های تأخیر بین Mistral Small و Large، سلول‌های زیر را اجرا کنید.

باید تفاوتی در زمان پاسخ‌دهی بین ۳ تا ۵ ثانیه مشاهده کنید. همچنین به طول و سبک پاسخ‌ها در برابر یک پرامپت یکسان توجه کنید.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## میسترال نِمو

در مقایسه با دو مدل دیگری که در این درس بررسی شدند، میسترال نِمو تنها مدلی است که به صورت رایگان و با مجوز Apache2 ارائه می‌شود.

این مدل به عنوان نسخه ارتقاء یافته مدل متن‌باز قبلی میسترال، یعنی Mistral 7B شناخته می‌شود.

برخی ویژگی‌های دیگر مدل نِمو عبارتند از:

- *توکن‌سازی کارآمدتر:* این مدل به جای tiktoken که رایج‌تر است، از توکنایزر Tekken استفاده می‌کند. این موضوع باعث می‌شود عملکرد بهتری در زبان‌ها و کدهای مختلف داشته باشد.

- *قابلیت فاین‌تیونینگ:* مدل پایه برای فاین‌تیونینگ در دسترس است. این ویژگی انعطاف‌پذیری بیشتری برای کاربردهایی که نیاز به فاین‌تیونینگ دارند فراهم می‌کند.

- *پشتیبانی بومی از فراخوانی توابع* - مانند Mistral Large، این مدل نیز برای فراخوانی توابع آموزش دیده است. این ویژگی باعث می‌شود یکی از اولین مدل‌های متن‌باز با این قابلیت باشد.


## میسترال نِمو

در مقایسه با دو مدل دیگری که در این درس بررسی شدند، میسترال نِمو تنها مدلی است که به صورت رایگان و با مجوز Apache2 ارائه می‌شود.

این مدل به عنوان نسخه ارتقاء یافته مدل متن‌باز قبلی میسترال، یعنی Mistral 7B شناخته می‌شود.

برخی ویژگی‌های دیگر مدل نِمو عبارتند از:

- *توکن‌سازی کارآمدتر:* این مدل به جای tiktoken که رایج‌تر است، از توکنایزر Tekken استفاده می‌کند. این موضوع باعث می‌شود عملکرد بهتری در زبان‌ها و کدهای مختلف داشته باشد.

- *قابلیت فاین‌تیونینگ:* مدل پایه برای فاین‌تیونینگ در دسترس است. این ویژگی انعطاف‌پذیری بیشتری برای کاربردهایی که نیاز به فاین‌تیونینگ دارند فراهم می‌کند.

- *پشتیبانی بومی از فراخوانی توابع* - مانند Mistral Large، این مدل نیز روی فراخوانی توابع آموزش دیده است. این ویژگی باعث می‌شود یکی از اولین مدل‌های متن‌باز با این قابلیت باشد.


### مقایسه توکنایزرها

در این نمونه، بررسی می‌کنیم که Mistral NeMo چگونه توکنیزه کردن را نسبت به Mistral Large انجام می‌دهد.

هر دو نمونه از یک پرامپت یکسان استفاده می‌کنند، اما باید متوجه شوید که NeMo تعداد توکن کمتری نسبت به Mistral Large برمی‌گرداند.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## یادگیری اینجا متوقف نمی‌شود، مسیر را ادامه دهید

بعد از اتمام این درس، مجموعه [یادگیری هوش مصنوعی مولد](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) را ببینید تا دانش خود را در زمینه هوش مصنوعی مولد بیشتر کنید!



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما برای دقت تلاش می‌کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطا یا نادقتی باشند. نسخه اصلی سند به زبان مادری آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، ترجمه حرفه‌ای انسانی توصیه می‌شود. ما هیچ مسئولیتی در قبال سوءتفاهم یا تفسیر نادرست ناشی از استفاده از این ترجمه نداریم.
